In [ ]:
# Import the required libraries

import pymongo
from datetime import datetime

import pandas as pd

In [ ]:
# Open a connection to the Mongo server
client = pymongo.MongoClient('mongodb://localhost:27351/')

In [ ]:
# Create a doctorwho database and a test_collection within it.
dw_db = client.doctorwho
tc = dw_db.test_collection
episodes = dw_db.episodes

# Activity 1

In [ ]:
tc.find_one({'birthyear': 1943})

# Activity 2

In [ ]:
# How many people were born in 1943?
tc.find({'birthyear': 1943}).count()

In [ ]:
# What are the names of people born in 1943?
list(tc.find({'birthyear': 1943}, {'name': 1, '_id': 0}))

# Activity 3

In [ ]:
for p in tc.find():
    if p['birthyear'] <= 1945:
        tc.update_one({'_id': p['_id']}, {'$set': {'age': 'old'}})
    else:
        tc.update_one({'_id': p['_id']}, {'$set': {'age': 'young'}})

pd.DataFrame(list(tc.find({}, {'_id': 0})))

# Activity 4

In [ ]:
for e in episodes.find():
    clean_parts = {}
    parts = e['Pt']
    if 'pt' in e:
        parts.update(e['pt'])
    for p in parts:
        k = p.strip()
        n = int(k[:2])
        if n not in clean_parts:
            clean_parts[n] = {'Number': n}
        if 'date' in k:
            d = datetime.strptime(parts[p], '%m/%d/%y')
            clean_parts[n]['Air date'] = d.replace(year=(d.year - 100))
        if 'viewers' in k:
            clean_parts[n]['Viewers'] = parts[p]
    assert len(clean_parts.values()) == e['No'][' of parts']
    episodes.update_one({'_id': e['_id']}, {'$set': {'Parts': list(clean_parts.values())}})
    
r = episodes.update_many({}, {'$unset': {'Pt': 1, 'pt': 1}})
r.matched_count, r.modified_count

In [ ]:
episodes.find_one({'Season': 1, 'Story ID': 4})